In [83]:
from PIL import Image, ExifTags
import os
import json
import datetime

In [ ]:
PICTURES_DIR = "resources/pictures"

In [84]:
# https://developer.here.com/blog/getting-started-with-geocoding-exif-image-metadata-in-python3
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()


def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in ExifTags.TAGS.items():
        if tag == "GPSInfo":
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in ExifTags.GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging


def get_decimal_from_dms(dms, ref):
    degrees = dms[0]
    minutes = dms[1] / 60.0
    seconds = dms[2] / 3600.0

    if ref in ["S", "W"]:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)


def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags["GPSLatitude"], geotags["GPSLatitudeRef"])
    lon = get_decimal_from_dms(geotags["GPSLongitude"], geotags["GPSLongitudeRef"])

    return (lat, lon)

def get_coordinates_for_image(img):
    exif = img._getexif()
    geotags = get_geotagging(exif)
    coordinates = get_coordinates(geotags)
    return coordinates

coordinates = get_coordinates_for_image(img)
coordinates


(52.54654, 13.35121)

In [85]:
def get_date_for_image(img):
    exif = {
        ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in ExifTags.TAGS and type(v) is not bytes
    }

    if "DateTimeOriginal" not in exif:
        return None
    dateRaw = exif["DateTimeOriginal"]
    date = datetime.datetime.strptime(dateRaw, "%Y:%m:%d %H:%M:%S").isoformat()
    return date


In [86]:
img_configs = []

# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk(PICTURES_DIR):
    path = root.split(os.sep)
    for file in files:
        img_path = os.path.join(root, file)
        img = Image.open(img_path)

        coordinates = get_coordinates_for_image(img)
        lat, lng = coordinates

        date = get_date_for_image(img)
        config = {
            "path": img_path,
            "location": {
                "lat": lat,
                "lng": lng
            },
            "date": date
        }
        img_configs += [config]


In [87]:
with open("picture-configs.json", "w") as f:
    json.dump(img_configs, f)